In [2]:
import pandas as pd
from binance.client import Client
import numpy as np

In [3]:
client = Client()

In [4]:
def getdata(symbol, start):
    frame = pd.DataFrame(client.get_historical_klines(symbol, '1h', start))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame.set_index('Time', inplace=True)
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame = frame.astype(float)
    return frame


In [5]:
df = getdata('BTCUSDT', '2023,01,01')

In [6]:
df ['chg'] = df.Close.pct_change()

In [7]:
df['12h_chg'] = df.chg.rolling(12).sum()

In [11]:
df.dropna(inplace=True)

In [12]:
df['buyprice'] = df.Open.shift(-1)

In [19]:
df

,Open,High,Low,Close,Volume,chg,12h_chg,buyprice
Time,,,,,,,,
2023-01-01 12:00:00,16556.66,16572.94,16548.58,16560.93,4150.48564,0.000258,0.001893,16561.29
2023-01-01 13:00:00,16561.29,16567.26,16539.05,16548.04,4379.68203,-0.000778,-0.000204,16547.61
2023-01-01 14:00:00,16547.61,16558.56,16540.85,16542.94,3294.99093,-0.000308,-0.000314,16542.94
2023-01-01 15:00:00,16542.94,16559.99,16533.68,16558.73,3220.93236,0.000954,0.001556,16558.73
2023-01-01 16:00:00,16558.73,16584.66,16558.00,16565.14,4962.13235,0.000387,0.002620,16565.13
...,...,...,...,...,...,...,...,...
2024-05-19 20:00:00,66034.01,66300.00,66019.99,66163.99,650.74941,0.001968,-0.014861,66164.00
2024-05-19 21:00:00,66164.00,66360.00,66000.00,66280.00,458.74226,0.001753,-0.013406,66280.00
2024-05-19 22:00:00,66280.00,66463.00,66252.00,66363.61,508.04173,0.001261,-0.013483,66363.61


In [20]:
in_position = False

profits = []

for index,row in df.iterrows():
    if not in_position:
        if row['12h_chg'] > 0.02:
            buyprice = row.buyprice
            in_position = True    
    if in_position:
        if row.High >= buyprice*1.01:
            sellprice = buyprice*1.01
            profit = (sellprice - buyprice)/buyprice
            profits.append(profit)
            in_position = False
            
        elif row.Low <= buyprice*0.99:
            sellprice = buyprice*0.99
            profit = (sellprice - buyprice)/buyprice
            profits.append(profit)
            in_position = False
            

In [23]:
(pd.Series(profits) + 1).prod() - 1

-0.5252776876730378

In [21]:
profits

[-0.010000000000000005,
 -0.00999999999999998,
 -0.009999999999999959,
 -0.009999999999999913,
 0.010000000000000035,
 0.010000000000000057,
 0.009999999999999988,
 -0.00999999999999998,
 0.010000000000000073,
 -0.009999999999999995,
 0.010000000000000089,
 -0.009999999999999946,
 0.010000000000000014,
 -0.01000000000000007,
 -0.010000000000000054,
 -0.010000000000000052,
 -0.009999999999999943,
 -0.010000000000000083,
 -0.010000000000000087,
 0.009999999999999997,
 -0.010000000000000089,
 0.010000000000000083,
 0.009999999999999988,
 -0.009999999999999993,
 0.01000000000000008,
 0.009999999999999941,
 -0.009999999999999985,
 -0.009999999999999943,
 -0.01000000000000007,
 0.01000000000000006,
 -0.009999999999999943,
 -0.010000000000000082,
 -0.009999999999999964,
 0.009999999999999983,
 0.00999999999999997,
 -0.010000000000000004,
 -0.01000000000000004,
 0.010000000000000042,
 -0.009999999999999959,
 0.009999999999999941,
 0.010000000000000075,
 -0.009999999999999953,
 -0.0100000000000

In [24]:
import os
os.getcwd()

'C:\\Users\\andre\\Documents\\Jupyter Python'